# <span style="font-width:bold; font-size: 3rem; color:#2656a3;">**Data Engineering and Machine Learning Operations in Business** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Feature Backfill</span>

## 🗒️ This notebook is divided into the following sections:
1. Load the data and process features
2. Connect to the Hopsworks feature store
3. Create feature groups and upload them to the feature store

## <span style='color:#2656a3'> ⚙️ Import of libraries and packages

First, we'll install the Python packages required for this notebook. We'll use the --quiet command after specifying the names of the libraries to ensure a silent installation process. Then, we'll proceed to import all the necessary libraries.

In [1]:
# Install of the packages for hopsworks
# !pip install -U hopsworks --quiet

In [2]:
# First we have to go one back in out directory so we can find the folder with our functions
%cd ..

# Now you can import the functions from the features folder. 
from features import electricity_prices, weater_measures # own function# This is functions we have created 

# Go back into the notebooks folder
%cd notebooks

/Users/tobiasmjensen/Documents/aau_bds/m5_data-engineering-and-mlops/exam_assigment/MLOPs-Assignment-
/Users/tobiasmjensen/Documents/aau_bds/m5_data-engineering-and-mlops/exam_assigment/MLOPs-Assignment-/notebooks


In [3]:
# Importing of the packages for the needed libraries for the Jupyter notebook
import pandas as pd
import requests
from datetime import datetime, timedelta

# Ignore warnings
import warnings 
warnings.filterwarnings('ignore')

## <span style="color:#2656a3;"> 💽 Load the historical data

The data you will use comes from three different sources:

- Electricity prices in Denmark per day from [Energinet](https://www.energidataservice.dk).
- Different meteorological observations from [Open meteo](https://www.open-meteo.com).
- Danish Calendar with the type if the date is a national holiday or not. This files is made manually by the group and is located in the "*data*" folder inside this repository.

### <span style="color:#2656a3;">💸 Electricity prices per day from Energinet
This first dataset is Electricity prices per day from Energinet/Dataservice. Here we use 

In [42]:
# Fetching historical electricity prices data from '2022-01-01' to today. 
# But then historical is set to True, today is not included in the data as it is not historical data
electricity_df = electricity_prices.electricity_prices(
    historical=True, 
    area=["DK1"], 
    start='2022-01-01', 
    #end='2023-12-31'
)

In [43]:
# Display the first 5 rows of the dataframe
electricity_df.head(5)

,timestamp,time,date,dk1_spotpricedkk_kwh,dk2_spotpricedkk_kwh
0,1640995200000000,2022-01-01 00:00:00,2022-01-01,0.37220,0.34654
1,1640998800000000,2022-01-01 01:00:00,2022-01-01,0.30735,0.30735
2,1641002400000000,2022-01-01 02:00:00,2022-01-01,0.32141,0.31367
3,1641006000000000,2022-01-01 03:00:00,2022-01-01,0.33806,0.32996
4,1641009600000000,2022-01-01 04:00:00,2022-01-01,0.28013,0.28013


In [6]:
# Display the last 5 rows of the dataframe
electricity_df.tail(5)

,timestamp,time,date,dk1_spotpricedkk_kwh
20200,1713726000000000,2024-04-21 19:00:00,2024-04-21,0.66671
20201,1713729600000000,2024-04-21 20:00:00,2024-04-21,0.71073
20202,1713733200000000,2024-04-21 21:00:00,2024-04-21,0.67947
20203,1713736800000000,2024-04-21 22:00:00,2024-04-21,0.66708
20204,1713740400000000,2024-04-21 23:00:00,2024-04-21,0.66000


In [7]:
# Showing the information for the electricity dataframe
electricity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20205 entries, 0 to 20204
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   timestamp             20205 non-null  int64         
 1   time                  20205 non-null  datetime64[ns]
 2   date                  20205 non-null  object        
 3   dk1_spotpricedkk_kwh  20205 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 631.5+ KB


### <span style="color:#2656a3;">☀️💨 Forecast Renewable Energy next day from Energinet

In [8]:
# Fetching historical forecast of renewable energy data from '2022-01-01' to today
# But then historical is set to True, today is not included in the data as it is not historical data
forecast_renewable_energy_df = electricity_prices.forecast_renewable_energy(
    historical=True, 
    area = ["DK1"],
    start= '2022-01-01', 
    #end='2023-12-31'
)

In [9]:
# Display the first 5 rows of the dataframe
forecast_renewable_energy_df.head(5)

,timestamp,time,date,dk1_offshore_wind_forecastcurrent_kwh,dk1_offshore_wind_forecastdayahead_kwh,dk1_offshore_wind_forecastintraday_kwh,dk1_onshore_wind_forecastcurrent_kwh,dk1_onshore_wind_forecastdayahead_kwh,dk1_onshore_wind_forecastintraday_kwh,dk1_solar_forecastcurrent_kwh,dk1_solar_forecastdayahead_kwh,dk1_solar_forecastintraday_kwh
0,1640995200000000,2022-01-01 00:00:00,2022-01-01,1.000417,0.830167,NaN,0.863000,0.793083,NaN,0.000022,0.0,NaN
1,1640998800000000,2022-01-01 01:00:00,2022-01-01,0.956583,0.870917,NaN,0.927167,0.798792,NaN,0.000002,0.0,NaN
2,1641002400000000,2022-01-01 02:00:00,2022-01-01,0.849500,0.832542,NaN,0.920333,0.790167,NaN,0.000000,0.0,NaN
3,1641006000000000,2022-01-01 03:00:00,2022-01-01,0.889542,0.646667,NaN,0.729125,0.716333,NaN,0.000000,0.0,NaN
4,1641009600000000,2022-01-01 04:00:00,2022-01-01,0.791667,0.569333,NaN,0.727667,0.593667,NaN,0.000000,0.0,NaN


In [10]:
# Display the last 5 rows of the dataframe
forecast_renewable_energy_df.tail(5)

,timestamp,time,date,dk1_offshore_wind_forecastcurrent_kwh,dk1_offshore_wind_forecastdayahead_kwh,dk1_offshore_wind_forecastintraday_kwh,dk1_onshore_wind_forecastcurrent_kwh,dk1_onshore_wind_forecastdayahead_kwh,dk1_onshore_wind_forecastintraday_kwh,dk1_solar_forecastcurrent_kwh,dk1_solar_forecastdayahead_kwh,dk1_solar_forecastintraday_kwh
20152,1713726000000000,2024-04-21 19:00:00,2024-04-21,0.993583,0.735417,0.770083,0.668417,0.474958,0.508625,0.228984,1.839621e-01,0.180932
20153,1713729600000000,2024-04-21 20:00:00,2024-04-21,0.997375,0.734583,0.769458,0.632792,0.407750,0.424458,0.041345,1.689833e-02,0.016666
20154,1713733200000000,2024-04-21 21:00:00,2024-04-21,0.702958,0.725542,0.746792,0.480250,0.396708,0.399042,0.001805,5.045830e-04,0.000485
20155,1713736800000000,2024-04-21 22:00:00,2024-04-21,0.734458,0.713042,0.714167,0.430000,0.389750,0.362917,0.000000,4.170000e-07,0.000000
20156,1713740400000000,2024-04-21 23:00:00,2024-04-21,0.782167,0.695208,0.675625,0.424917,0.360583,0.319542,0.000000,0.000000e+00,0.000000


In [11]:
# Showing the information for the forecast_renewable_energy dataframe
forecast_renewable_energy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20157 entries, 0 to 20156
Data columns (total 12 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   timestamp                               20157 non-null  int64         
 1   time                                    20157 non-null  datetime64[ns]
 2   date                                    20157 non-null  object        
 3   dk1_offshore_wind_forecastcurrent_kwh   20148 non-null  float64       
 4   dk1_offshore_wind_forecastdayahead_kwh  20085 non-null  float64       
 5   dk1_offshore_wind_forecastintraday_kwh  14175 non-null  float64       
 6   dk1_onshore_wind_forecastcurrent_kwh    20148 non-null  float64       
 7   dk1_onshore_wind_forecastdayahead_kwh   20109 non-null  float64       
 8   dk1_onshore_wind_forecastintraday_kwh   14175 non-null  float64       
 9   dk1_solar_forecastcurrent_kwh           20157 non-

### <span style="color:#2656a3;"> 🌤 Weather measurements from Open Meteo

Burde have enddate 2023-12-31. url = ("https://archive-api.open-meteo.com/v1/archive?latitude=57.048&longitude=9.9187&start_date=2022-01-01&end_date=2023-12-31&hourly=temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m")

#### <span style="color:#2656a3;"> 🕰️ Historical Weater Measures

In [12]:
# Fetching historical electricity prices data
historical_weather_df = weater_measures.historical_weater_measures(
    historical=True, 
    start = '2022-01-01', 
    #end = '2023-12-31'
)

In [13]:
# Display the first 5 rows of the dataframe
historical_weather_df.head(5)

,timestamp,date,time,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
0,1640995200000000,2022-01-01,2022-01-01 00:00:00,6.7,100.0,0.0,0.0,0.0,3.0,100.0,16.2,36.0
1,1640998800000000,2022-01-01,2022-01-01 01:00:00,6.6,100.0,0.0,0.0,0.0,3.0,100.0,16.2,30.2
2,1641002400000000,2022-01-01,2022-01-01 02:00:00,6.7,99.0,0.0,0.0,0.0,3.0,100.0,15.5,30.6
3,1641006000000000,2022-01-01,2022-01-01 03:00:00,6.7,100.0,0.0,0.0,0.0,3.0,100.0,12.7,28.8
4,1641009600000000,2022-01-01,2022-01-01 04:00:00,6.7,99.0,0.0,0.0,0.0,3.0,100.0,10.6,23.8


In [46]:
# Display the last 5 rows of the dataframe
historical_weather_df.tail(5)

,timestamp,date,time,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
20203,1713726000000000,2024-04-21,2024-04-21 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20204,1713729600000000,2024-04-21,2024-04-21 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20205,1713733200000000,2024-04-21,2024-04-21 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20206,1713736800000000,2024-04-21,2024-04-21 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20207,1713740400000000,2024-04-21,2024-04-21 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Showing the information for the weather dataframe
historical_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20208 entries, 0 to 20207
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   timestamp             20208 non-null  int64         
 1   date                  20208 non-null  object        
 2   time                  20208 non-null  datetime64[ns]
 3   temperature_2m        20184 non-null  float64       
 4   relative_humidity_2m  20184 non-null  float64       
 5   precipitation         20184 non-null  float64       
 6   rain                  20184 non-null  float64       
 7   snowfall              20184 non-null  float64       
 8   weather_code          20184 non-null  float64       
 9   cloud_cover           20184 non-null  float64       
 10  wind_speed_10m        20184 non-null  float64       
 11  wind_gusts_10m        20184 non-null  float64       
dtypes: datetime64[ns](1), float64(9), int64(1), object(1)
memory usage: 2.0+ M

#### <span style="color:#2656a3;"> 🌈 Weater Forecast

In [16]:
# Fetching historical electricity prices data
weather_forecast_df = weater_measures.forecast_weater_measures(
    forecast_length=5
)

In [17]:
# Display the first 5 rows of the dataframe
weather_forecast_df.head(5)

,timestamp,date,time,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
0,1713744000000000,2024-04-22,2024-04-22 00:00:00,3.5,60,0.0,0.0,0.0,3,98,6.8,14.0
1,1713747600000000,2024-04-22,2024-04-22 01:00:00,3.3,63,0.0,0.0,0.0,3,100,6.5,10.4
2,1713751200000000,2024-04-22,2024-04-22 02:00:00,3.0,61,0.0,0.0,0.0,3,100,7.6,13.0
3,1713754800000000,2024-04-22,2024-04-22 03:00:00,2.6,61,0.0,0.0,0.0,2,75,7.9,14.8
4,1713758400000000,2024-04-22,2024-04-22 04:00:00,2.1,60,0.0,0.0,0.0,2,53,9.0,14.4


In [18]:
# Display the last 5 rows of the dataframe
weather_forecast_df.tail(5)

,timestamp,date,time,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
163,1714330800000000,2024-04-28,2024-04-28 19:00:00,11.6,72,0.0,0.0,0.0,1,78,12.1,24.1
164,1714334400000000,2024-04-28,2024-04-28 20:00:00,10.7,74,0.0,0.0,0.0,1,57,11.6,23.4
165,1714338000000000,2024-04-28,2024-04-28 21:00:00,10.0,77,0.0,0.0,0.0,1,35,11.0,22.3
166,1714341600000000,2024-04-28,2024-04-28 22:00:00,9.8,80,0.0,0.0,0.0,3,57,10.7,21.6
167,1714345200000000,2024-04-28,2024-04-28 23:00:00,9.8,83,0.0,0.0,0.0,3,78,11.2,21.2


In [19]:
# Showing the information for the weather dataframe
weather_forecast_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   timestamp             168 non-null    int64         
 1   date                  168 non-null    object        
 2   time                  168 non-null    datetime64[ns]
 3   temperature_2m        168 non-null    float64       
 4   relative_humidity_2m  168 non-null    int64         
 5   precipitation         168 non-null    float64       
 6   rain                  168 non-null    float64       
 7   snowfall              168 non-null    float64       
 8   weather_code          168 non-null    int64         
 9   cloud_cover           168 non-null    int64         
 10  wind_speed_10m        168 non-null    float64       
 11  wind_gusts_10m        168 non-null    float64       
dtypes: datetime64[ns](1), float64(6), int64(4), object(1)
memory usage: 15.9+ KB


### <span style="color:#2656a3;"> 🗓️ Calendar of Danish workdays and holidays 

In [20]:
# Read csv file with calender
calender_df = pd.read_csv('https://raw.githubusercontent.com/Camillahannesbo/MLOPs-Assignment-/main/data/calendar_incl_holiday.csv', delimiter=';', usecols=['date', 'type'])
 
# Display the DataFrame
calender_df.head()

,date,type
0,01/01/2022,Not a Workday
1,02/01/2022,Not a Workday
2,03/01/2022,Workday
3,04/01/2022,Workday
4,05/01/2022,Workday


In [21]:
# Formatting the date column to 'YYYY-MM-DD' dateformat
calender_df["date"] = calender_df["date"].map(lambda x: datetime.strptime(x, '%d/%m/%Y').strftime("%Y-%m-%d"))

In [22]:
# Display the first 5 rows of the dataframe
calender_df.head(5)

,date,type
0,2022-01-01,Not a Workday
1,2022-01-02,Not a Workday
2,2022-01-03,Workday
3,2022-01-04,Workday
4,2022-01-05,Workday


In [23]:
# Display the last 5 rows of the dataframe
calender_df.tail(5)

,date,type
1091,2024-12-27,Workday
1092,2024-12-28,Not a Workday
1093,2024-12-29,Not a Workday
1094,2024-12-30,Workday
1095,2024-12-31,Workday


In [24]:
# Showing the information for the calender dataframe
calender_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    1096 non-null   object
 1   type    1096 non-null   object
dtypes: object(2)
memory usage: 17.3+ KB


## <span style="color:#2656a3;"> 📡 Connecting to Hopsworks Feature Store

First we will connect to Hopsworks Feature Store so we can access and create Feature Groups.
Feature groups can also be used to define a namespace for features. For instance, in a real-life setting you would likely want to experiment with different window lengths. In that case, you can create feature groups with identical schema for each window length. 

Before you can create a feature group you need to connect to our feature store.

In [25]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/554133
Connected. Call `.close()` to terminate connection gracefully.


### <span style="color:#2656a3;"> 🪄 Creating Feature Groups

When creating a feature group, you must name it and designate a primary key. Additionally, it's helpful to include a description of the feature group's contents and a version number; if not defined, it will default to `1`. 

We've configured `online_enabled` as `True` to enable the feature group to be read via the Online API for a Feature View.

In [26]:
# Creating the feature group for the weater data
weather_fg = fs.get_or_create_feature_group(
    name="weather_measurements",
    version=1,
    description="Weather measurements from Open Meteo API",
    primary_key=["date"],
    event_time="timestamp",
    online_enabled=True,
)

By now, you've only outlined metadata for the feature group. There's no data stored, nor is there a defined schema for it. To establish persistence for the feature group, you'll need to populate it with its associated data using the `insert` function

In [27]:
# Inserting the weather_df into the feature group named weather_fg
weather_fg.insert(historical_weather_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/554133/fs/549956/fg/743765


Uploading Dataframe: 100.00% |██████████| Rows 20208/20208 | Elapsed Time: 00:08 | Remaining Time: 00:00


Launching job: weather_measurements_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/554133/jobs/named/weather_measurements_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1769922d0>, None)

We make a descriptions for each feature we put into the feature group. In this way we are adding more information and documentation to the user

In [28]:
weather_feature_descriptions = [
    {"name": "timestamp", "description": "Timestamp for the event_time"},
    {"name": "date", "description": "Date of the weather measurement"},
    {"name": "time", "description": "Time of the weather measurement"},
    {"name": "temperature_2m", "description": "Temperature at 2m above ground"},
    {"name": "relative_humidity_2m", "description": "Relative humidity at 2m above ground"},
    {"name": "precipitation", "description": "Precipitation"},
    {"name": "rain", "description": "Rain"},
    {"name": "snowfall", "description": "Snowfall"},   
    {"name": "weather_code", "description": "Weather code"},   
    {"name": "cloud_cover", "description": "Cloud cover"},   
    {"name": "wind_speed_10m", "description": "Wind speed at 10m above ground"},   
    {"name": "wind_gusts_10m", "description": "Wind gusts at 10m above ground"},   
]

for desc in weather_feature_descriptions: 
    weather_fg.update_feature_description(desc["name"], desc["description"])

We replicate the process for both the `electricity_fg`, `forecast_renewable_energy_fg` and `danish_holidays_fg` by establishing feature groups and inserting the dataframes into their respective feature groups.

In [29]:
# Creating the feature group for the electricity prices
electricity_fg = fs.get_or_create_feature_group(
    name="electricity_prices",
    version=1,
    description="Electricity prices from Energidata API",
    primary_key=["date"],
    online_enabled=True,
    event_time="timestamp",
)

In [30]:
# Inserting the electricity_df into the feature group named electricity_fg
electricity_fg.insert(electricity_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/554133/fs/549956/fg/744778


Uploading Dataframe: 100.00% |██████████| Rows 20205/20205 | Elapsed Time: 00:08 | Remaining Time: 00:00


Launching job: electricity_prices_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/554133/jobs/named/electricity_prices_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x17692a0d0>, None)

In [31]:
electricity_feature_descriptions = [
    {"name": "timestamp", "description": "Timestamp for the event_time"},
    {"name": "date", "description": "Date of the electricity measurement"},
    {"name": "time", "description": "Time of the electricity measurement"},
    {"name": "dk1_spotpricedkk_kwh", "description": "Spot price in DKK per KWH"}, 
]

for desc in electricity_feature_descriptions: 
    electricity_fg.update_feature_description(desc["name"], desc["description"])

In [32]:
# Creating the feature group for the electricity prices
forecast_renewable_energy_fg = fs.get_or_create_feature_group(
    name="forecast_renewable_energy",
    version=1,
    description="Forecast on Renewable Energy on ForecastType from Energidata API",
    primary_key=["date"],
    online_enabled=True,
    event_time="timestamp",
)

In [33]:
# Inserting the electricity_df into the feature group named electricity_fg
forecast_renewable_energy_fg.insert(forecast_renewable_energy_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/554133/fs/549956/fg/744779


Uploading Dataframe: 100.00% |██████████| Rows 20157/20157 | Elapsed Time: 00:09 | Remaining Time: 00:00


Launching job: forecast_renewable_energy_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/554133/jobs/named/forecast_renewable_energy_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x17693bcd0>, None)

In [36]:
forecast_renewable_energy_feature_descriptions = [
    {"name": "timestamp", "description": "Timestamp for the event_time"},
    {"name": "date", "description": "Date"},
    {"name": "time", "description": "Time for the event_time"},
    {"name": "dk1_offshore_wind_forecastdayahead_kwh", "description": "Forecast for the next day is published at 18:00 Danish time zone."},
    {"name": "dk1_offshore_wind_forecastintraday_kwh", "description": "The forecast for the coming day at 6am Danish time zone"},
    {"name": "dk1_offshore_wind_forecastcurrent_kwh", "description": "The forecast valid for the current time"},
]

for desc in forecast_renewable_energy_feature_descriptions: 
    forecast_renewable_energy_fg.update_feature_description(desc["name"], desc["description"])

In [37]:
# Creating the feature group for the danish holidays
danish_holidays_fg = fs.get_or_create_feature_group(
    name="danish_holidays",
    version=1,
    description="Danish holidays calendar.",
    online_enabled=True,
    primary_key=["date"],
)

In [38]:
# Inserting the calender_df into the feature group named danish_holidays_fg
danish_holidays_fg.insert(calender_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/554133/fs/549956/fg/742740


Uploading Dataframe: 100.00% |██████████| Rows 1096/1096 | Elapsed Time: 00:08 | Remaining Time: 00:00


Launching job: danish_holidays_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/554133/jobs/named/danish_holidays_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x17693a950>, None)

In [39]:
danish_holidays_feature_descriptions = [
    {"name": "date", "description": "Date in the calendar"},
    {"name": "type", "description": "Holyday or not holyday"},
]

for desc in danish_holidays_feature_descriptions: 
    danish_holidays_fg.update_feature_description(desc["name"], desc["description"])

---
## <span style="color:#2656a3;">⏭️ **Next:** Part 02: Feature Pipeline </span>

In the next notebook, you will be generating new data for the Feature Groups.